# Wrangle Exercises

In [1]:
import pyspark
from pyspark.sql.functions import *

spark = pyspark.sql.SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/20 15:33:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/20 15:33:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Data Prep

In [3]:
df = spark.read.csv('case.csv', header=True, inferSchema=True)

In [4]:
# Rename column
df = df.withColumnRenamed('SLA_due_date', 'case_due_date')

# Convert to better data types
df = (
    df.withColumn('case_late', col('case_late') == 'YES')
    .withColumn('case_closed', col('case_closed') == 'YES')
)
df = df.withColumn('council_district', format_string('%04d', col('council_district')))
df = (
    df.withColumn('case_opened_date', to_timestamp(col('case_opened_date'), 'M/d/yy H:mm'))
    .withColumn('case_closed_date', to_timestamp(col('case_closed_date'), 'M/d/yy H:mm'))
    .withColumn('case_due_date', to_timestamp(col('case_due_date'), 'M/d/yy H:mm'))
)

# Cleanup text data
df = df.withColumn('request_address', lower(trim(col('request_address'))))
# Extract zipcode
df = df.withColumn('zipcode', regexp_extract(col('request_address'), r'\d+$', 0))

# Create a `case_lifetime` feature
df = (
    df.withColumn('case_age', datediff(current_timestamp(), 'case_opened_date'))
    .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
    .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
    .drop('case_age', 'days_to_closed')
)

# Join departments and sources
depts = spark.read.csv('dept.csv', header=True, inferSchema=True)
sources = spark.read.csv('source.csv', header=True, inferSchema=True)

df = df.join(depts, 'dept_division', 'left').join(sources, 'source_id', 'left')

# # Train Test Split
# train, test = df.randomSplit([.8, .2], seed=123)
# train, validate, test = df.randomSplit([.7, .15, .15], seed=123)

In case we want to query our dataframe with Spark SQL:

In [5]:
df.createOrReplaceTempView('df')

## Initial Exploration Questions

In [6]:
# How old is the latest (in terms of days past SLA) currently open issue?
# How long has the oldest (in terms of days since opened) currently opened issue been open?
spark.sql('''
SELECT DATEDIFF(current_timestamp, case_due_date) AS days_past_due
FROM df
WHERE NOT case_closed
ORDER BY days_past_due DESC
LIMIT 15
''').show()

+-------------+
|days_past_due|
+-------------+
|         1949|
|         1949|
|         1949|
|         1948|
|         1946|
|         1942|
|         1942|
|         1941|
|         1940|
|         1940|
|         1936|
|         1935|
|         1935|
|         1935|
|         1932|
+-------------+



In [7]:
# How many Stray Animal cases are there?
df.filter(df.service_request_type == 'Stray Animal').count()

27361

26760?

In [9]:
(
    df.groupBy('service_request_type')
    .count()
    .filter(expr('service_request_type == "Stray Animal"'))
    .show()
)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|        Stray Animal|27361|
+--------------------+-----+



In [10]:
# How many service requests that are assigned to the Field Operations department (dept_division)
# are not classified as "Officer Standby" request type (service_request_type)?
(
    df.filter(df.dept_division == 'Field Operations')
    .filter(df.service_request_type != 'Officer Standby')
    .count()
)

116295

In [15]:
# Another way to do it
(
    df.filter(expr("dept_division == 'Field Operations'"))
    .groupBy('service_request_type')
    .count()
    .show(truncate=False)
)

+-------------------------------+-----+
|service_request_type           |count|
+-------------------------------+-----+
|Animal Permits Request         |3083 |
|Injured Animal(Critical)       |9779 |
|Officer Standby                |3067 |
|Animal Bite(Non-Critical)      |4783 |
|Stray Animal                   |27361|
|Trapped/Confined Animal        |11605|
|City Council Animal Request    |365  |
|Aggressive Animal(Non-Critical)|25492|
|Animal Bite(Critical)          |717  |
|Public Nuisance(Own Animal)    |10969|
|Aggressive Animal(Critical)    |5280 |
|Animal Cruelty(Critical)       |3009 |
|Animal Neglect                 |13851|
|Spay/Neuter Request Response   |1    |
+-------------------------------+-----+



In [13]:
# Another way to do it
(
    df.filter(expr("dept_division == 'Field Operations'"))
    .groupBy(expr('service_request_type != "Officer Standby"'))
    .show()
)

+----------------------------------------------+------+
|(NOT (service_request_type = Officer Standby))| count|
+----------------------------------------------+------+
|                                          true|116295|
|                                         false|  3067|
+----------------------------------------------+------+



Fantasque Sans Mono

```
brew search fantasque
```

In [ ]:
# Convert the council_district column to a string column.

# Already done in the data prep

In [16]:
# Extract the year from the case_closed_date column.
df.select('case_closed_date', year('case_closed_date')).show(5)

+-------------------+----------------------+
|   case_closed_date|year(case_closed_date)|
+-------------------+----------------------+
|2018-01-01 12:29:00|                  2018|
|2018-01-03 08:11:00|                  2018|
|2018-01-02 07:57:00|                  2018|
|2018-01-02 08:13:00|                  2018|
|2018-01-01 13:29:00|                  2018|
+-------------------+----------------------+
only showing top 5 rows



In [17]:
# Convert num_days_late from days to hours in new columns num_hours_late.
(
    df.withColumn('num_hours_late', df.num_days_late * 24)
    .select('num_days_late', 'num_hours_late')
    .show()
)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
|       -29.74398148| -713.8555555199999|
|       -14.70673611|      -352.96166664|
|       -14.70662037|      -352.95888888|
|       -14.70662037|      -352.95888888|
|       -14.70649306|      -352.95583344|
|       -14.70649306|      -352.95583344|
|       -14.70636574|      -352.95277776|
|          -14.70625|-352.95000000000005|
|       -14.70636574|      -352.95277776|
|       -14.70623843|-352.94972232000003|
|-14.705891199999998|-352.94138879999997|
|       -14.70600694|      -352.94416656|
|       -14.70576389|      -352.93833336|
|       -14.70576389|      -352.93833336|
|       -14.70564815|       -352.9355556|
+-------------------+-------------

In [ ]:
# Join the case data with the source and department data.

# already joined in the data prep

In [18]:
# Are there any cases that do not have a request source?
# are there any null values for source_id?
(
    df.select(df.source_id.isNull().cast('int').alias('is_null'))
    .agg(sum('is_null'))
    .show()
)

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



In [19]:
df.filter(col('source_id').isNull()).show(vertical=True)

(0 rows)



In [20]:
# What are the top 10 service request types in terms of number of requests?
(
    df.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



In [21]:
# What are the top 10 service request types in terms of average days late?
# - just the late cases
# - for the late cases:
#   - what is the average number of days late by request type?
(
    df.where('case_late') # just the rows where case_late == true
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.20517608494276|262    |
|Structure/Housing Maintenance         |190.20707698509807|51     |
|Donation Container Enforcement        |171.09115313942615|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|4      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



In [22]:
# Does number of days late depend on department?
(
    df.filter('case_late')
    .groupby('dept_name')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)
)

+-------------------------+---------+------------+
|dept_name                |days_late|n_cases_late|
+-------------------------+---------+------------+
|Metro Health             |6.5      |854         |
|Solid Waste Management   |7.1      |33729       |
|Trans & Cap Improvements |10.7     |5529        |
|Parks and Recreation     |22.4     |3810        |
|Animal Care Services     |23.4     |23751       |
|Code Enforcement Services|48.1     |25467       |
|Development Services     |67.2     |840         |
|Customer Service         |88.2     |2035        |
|null                     |210.9    |132         |
+-------------------------+---------+------------+



In [ ]:
df.groupby('dept_name').count().show(truncate=False)

In [24]:
# How do number of days late depend on department and request type?
(
    df.filter("case_closed")
    .filter("case_late")
    .groupby("standardized_dept_name", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases"))
    .withColumn("days_late", round(col("days_late"), 1))
    .sort(col('standardized_dept_name'), col('days_late'))
    .show(40, truncate=False)
)

+----------------------+--------------------------------------+---------+-------+
|standardized_dept_name|service_request_type                  |days_late|n_cases|
+----------------------+--------------------------------------+---------+-------+
|Animal Care Services  |Injured Animal(Critical)              |0.1      |578    |
|Animal Care Services  |Officer Standby                       |0.1      |97     |
|Animal Care Services  |Animal Bite(Critical)                 |0.2      |217    |
|Animal Care Services  |Animal Cruelty(Critical)              |0.2      |389    |
|Animal Care Services  |Trapped/Confined Animal               |0.8      |5806   |
|Animal Care Services  |Animal Bite(Non-Critical)             |4.7      |133    |
|Animal Care Services  |City Council Animal Request           |9.2      |31     |
|Animal Care Services  |Aggressive Animal(Critical)           |21.5     |4084   |
|Animal Care Services  |Animal Neglect                        |30.0     |4630   |
|Animal Care Ser

In [26]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

import wrangle

df = wrangle.wrangle_311(spark)

[wrangle.py] reading case.csv


[wrangle.py] handling data types
[wrangle.py] parsing dates
[wrangle.py] adding features


[wrangle.py] joining departments


In [ ]:
# ... df